In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from bq_link import get_bq_link
import os
from configparser import ConfigParser
import upload

In [2]:
def get_config():
    config = ConfigParser()
    config.read(os.path.join("C:/Users/andres.mireles_ebury/Desktop/Projects/FX Exposure/fx_exposure/config.ini"))

    return config


In [3]:
config = get_config()
bq_client, _ = get_bq_link(config)

In [4]:
url = "https://docs.google.com/spreadsheets/d/11eRqnaQC_Mmr9zL73B6kD9oCQdNI4LVuyDEuAfBUt8Q/edit#gid=2039940707"

In [5]:
from gspread_pandas import Spread, conf

# below_the_line = Spread(url, creds=bq_client.credentials).sheet_to_df().reset_index()
below_the_line = Spread(url, creds=bq_client.credentials)

In [6]:
sheets = pd.DataFrame()

In [7]:
for sheet in below_the_line.sheets:
    sheet_df = below_the_line.sheet_to_df(sheet=sheet.title,index=None)

    date_range = pd.date_range(start=f'{sheet.title}-01', freq='D', periods=pd.Timestamp(sheet.title).days_in_month)
    
    for date in date_range:

        sheet_df["date"] = date
        sheets = pd.concat([sheets,sheet_df])
        

In [8]:
sheets["date"] = pd.to_datetime(sheets["date"])

In [9]:
sheets.columns = ["currency","below_the_line_gbp","balance_date"]

In [10]:
sheets["below_the_line_gbp"] = sheets["below_the_line_gbp"].str.replace(",","").astype(float)
sheets["balance_date"] = sheets["balance_date"].astype("str")

In [11]:
sheets.dtypes

currency               object
below_the_line_gbp    float64
balance_date           object
dtype: object

In [12]:
sheets.sort_values("currency")

,currency,below_the_line_gbp,balance_date
0,AED,-4028893.00,2024-03-08
3,AED,-3016538.35,2023-12-22
0,AED,-1220916.00,2024-05-14
3,AED,-3016538.35,2023-12-21
0,AED,-4028893.00,2024-03-22
...,...,...,...
49,ZMW,6981.33,2023-12-30
28,ZMW,63385.48,2024-01-20
105,ZMW,331354.00,2024-05-28
28,ZMW,63385.48,2024-01-25


In [13]:
upload.upload_table(
    sheets,
    "below_the_line_monthly",
    bq_client,
    config,
)

Upload to BQ done! portfolio_metrics.below_the_line_monthly


In [14]:
19545

19545